In [1]:
from keras.models import Sequential,Model
from keras.layers import Input,Flatten,Dense,Dropout,GlobalAveragePooling2D,Conv2D,MaxPooling2D
import numpy as np
from keras import backend as K
import os

# Disable GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Using TensorFlow backend.


In [2]:
imagedir = "Datasets/malimg_paper_dataset_imgs"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names

os.chdir(cur_dir)

In [4]:
list_fams

['Adialer.C',
 'Agent.FYI',
 'Allaple.A',
 'Allaple.L',
 'Alueron.gen!J',
 'Autorun.K',
 'C2LOP.gen!g',
 'C2LOP.P',
 'Dialplatform.B',
 'Dontovo.A',
 'Fakerean',
 'Instantaccess',
 'Lolyda.AA1',
 'Lolyda.AA2',
 'Lolyda.AA3',
 'Lolyda.AT',
 'Malex.gen!J',
 'Obfuscator.AD',
 'Rbot!gen',
 'Skintrim.N',
 'Swizzor.gen!E',
 'Swizzor.gen!I',
 'VB.AT',
 'Wintrim.BX',
 'Yuner.A']

In [5]:
# Creating base model (VGG16 convolutional layers)
img_shape = (224, 224, 3)                                                                                                                                               
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=img_shape))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

In [6]:
vgg16weights = "vgg16_weights_tf_dim_ordering_tf_kernels.h5"
model.load_weights(vgg16weights, by_name=True)

In [7]:
# Stacking top model (fc and softmax layers)
model.add(Flatten())                                                
model.add(Dense(160, activation='relu', name='fc1'))
model.add(Dropout(0.4))                
model.add(Dense(160, activation='relu', name='fc2'))
model.add(Dropout(0.4))              
model.add(Dense(25, activation='softmax', name='predictions'))                             

In [8]:
top_weigths = "top_model-weights-max_val_acc-mlp-vgg16-2layers-160neurons-relu-0.4dropout-Adam-500epochs.h5"
model.load_weights(top_weigths, by_name=True)

In [9]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
from quiver_engine import server

In [13]:
server.launch(model, temp_folder='QuiverOutput', input_folder='QuiverInput', port=5001)
#quiver_engine.server.launch(model, classes=['cat','dog'], input_folder='./imgs')

Starting webserver from: /usr/local/lib/python3.5/dist-packages/quiver_engine


::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET / HTTP/1.1" 304 235 0.016315
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /app.c1d3d21cfba8c1aabed6806773c8f0ba.css HTTP/1.1" 304 236 0.003303
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /app.14b8092d6ea379715007.js HTTP/1.1" 304 239 0.003131
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /vendor.280c5ce1d78077ee8d66.js HTTP/1.1" 304 237 0.003050
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /model HTTP/1.1" 200 17605 0.013606
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /inputs HTTP/1.1" 200 2886 0.002931
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /input-file/Swizzor.gen!I-000beddf5e7f49a1718bfd5586e93665.png HTTP/1.1" 304 242 0.003224
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /input-file/C2LOP.P-00052e46dc77c6e62faa608f656b4eef.png HTTP/1.1" 304 244 0.003111
::ffff:200.144.117.133 - - [2017-09-27 15:57:44] "GET /input-file/Lolyda.AT-00036955722f0041d91cf4ce4f70e94b.

::ffff:200.144.117.133 - - [2017-09-27 15:57:56] "GET /predict/Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png HTTP/1.1" 500 488 0.016400
/usr/local/lib/python3.5/dist-packages/quiver_engine/layer_result_generators.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("bl..., outputs=Tensor("bl...)`
  output=model.get_layer(layer_name).output
::ffff:200.144.117.133 - - [2017-09-27 15:57:58] "GET /layer/block1_conv1/Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png HTTP/1.1" 200 4786 1.839879
::ffff:200.144.117.133 - - [2017-09-27 15:57:58] "GET /temp-file/block1_conv1_0_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 32588 0.003544
::ffff:200.144.117.133 - - [2017-09-27 15:57:58] "GET /temp-file/block1_conv1_5_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 4480 0.003223
::ffff:200.144.117.133 - - [2017-09-27 15:57:58] "GET /temp-file/block1_conv1_4_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 32474 0.003206
::

::ffff:200.144.117.133 - - [2017-09-27 15:58:11] "GET /layer/block1_conv2/Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png HTTP/1.1" 200 4786 1.781895
::ffff:200.144.117.133 - - [2017-09-27 15:58:11] "GET /temp-file/block1_conv2_1_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 34146 0.003781
::ffff:200.144.117.133 - - [2017-09-27 15:58:11] "GET /temp-file/block1_conv2_6_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 33497 0.003313
::ffff:200.144.117.133 - - [2017-09-27 15:58:11] "GET /temp-file/block1_conv2_7_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 42311 0.003528
::ffff:200.144.117.133 - - [2017-09-27 15:58:11] "GET /temp-file/block1_conv2_8_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 41748 0.003389
::ffff:200.144.117.133 - - [2017-09-27 15:58:11] "GET /temp-file/block1_conv2_9_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 33845 0.003223
::ffff:200.144.117.133 - - [2017-09-27 15:58:11] "GET /te

::ffff:200.144.117.133 - - [2017-09-27 15:58:18] "GET /layer/block1_conv1/Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png HTTP/1.1" 200 4786 1.971418
::ffff:200.144.117.133 - - [2017-09-27 15:58:23] "GET /layer/block1_conv2/Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png HTTP/1.1" 200 4786 2.061090
::ffff:200.144.117.133 - - [2017-09-27 15:58:26] "GET /layer/block1_pool/Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png HTTP/1.1" 200 4722 0.505786
::ffff:200.144.117.133 - - [2017-09-27 15:58:26] "GET /temp-file/block1_pool_0_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 12613 0.003674
::ffff:200.144.117.133 - - [2017-09-27 15:58:26] "GET /temp-file/block1_pool_6_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 11868 0.004256
::ffff:200.144.117.133 - - [2017-09-27 15:58:26] "GET /temp-file/block1_pool_7_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 11390 0.003232
::ffff:200.144.117.133 - - [2017-09-27 15:58:26] "GET /temp-file/block1_pool_8_Fake

...

::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_270_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 396 0.005007
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_271_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 423 0.003289
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_272_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 487 0.003052
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_273_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 402 0.003151
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_274_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 429 0.003187
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_275_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 470 0.003154
::ffff:200.144.117.133 - - [2017-09-27 16:43:0

::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_338_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 503 0.005169
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_339_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 841 0.003159
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_340_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 643 0.003162
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_341_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 620 0.003143
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_342_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 764 0.003124
::ffff:200.144.117.133 - - [2017-09-27 16:43:01] "GET /temp-file/block4_conv3_343_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 893 0.003142
::ffff:200.144.117.133 - - [2017-09-27 16:43:0

::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_405_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 477 0.003084
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_406_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 398 0.003038
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_407_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 475 0.003070
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_408_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 410 0.003294
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_409_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 675 0.003169
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_410_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 483 0.003165
::ffff:200.144.117.133 - - [2017-09-27 16:43:0

::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_470_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 437 0.004588
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_471_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 503 0.003641
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_472_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 440 0.003087
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_473_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 394 0.003189
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_474_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 619 0.003120
::ffff:200.144.117.133 - - [2017-09-27 16:43:02] "GET /temp-file/block4_conv3_475_Fakerean-0005f5bdae46ea2d6d502b04fd2d0d5f.png.png HTTP/1.1" 200 420 0.003196
::ffff:200.144.117.133 - - [2017-09-27 16:43:0

KeyboardInterrupt: 